## Implementação de funções auxiliares e importação de bibliotecas importantes

In [240]:
from random import randint, seed
seed(17)

# Função de cálculo da distância de uma lista de sequências de viagens a visitar
def calc_tour_length(solution, distance_matrix):
    if len(solution) == 0:
        return 0
    if len(solution) == 1:
        return distance_matrix[0][solution[0]-1] + distance_matrix[solution[0]-1][0]
    start_index = solution[0]
    end_index = solution[-1]
    total = distance_matrix[0][start_index-1] + distance_matrix[end_index-1][0]
    for i in range(1, len(solution)):
        total += distance_matrix[solution[i-1]-1][solution[i]-1]
    return total

# Função de busca local com a heurística 2-opt 
def local_search_two_opt(solution,obj_function,distance_matrix):
    sol_copy = solution.copy()
    obj_function_copy = obj_function 
    route = sol_copy
    obj_function_route = obj_function
    size_of_route = len(route)
    if size_of_route > 2:
        for point_a in range(size_of_route - 2):
            for point_b in range(point_a+2,size_of_route):
                new_sol, new_obj_function = two_opt_intraroute(route,point_a,point_b,obj_function_route,distance_matrix)

                if new_obj_function < obj_function:
                    return new_sol, new_obj_function
                sol_copy = solution.copy()
    return solution, obj_function

# Função de modificação determinística de uma solução pela heurística 2-opt
def two_opt_intraroute(route_vector,point_a,point_b,obj_function, distance_matrix):
    #point_a, point_b = min([point_a, point_b]), max([point_a, point_b])
    new_vector = [1] + route_vector[:point_a] + route_vector[point_a:point_b][::-1] + route_vector[point_b:] + [1]
    point_a += 1
    point_b += 1
    obj_function = obj_function - distance_matrix[new_vector[point_a-1]-1][new_vector[point_b-1]-1] 
    obj_function = obj_function - distance_matrix[new_vector[point_a]-1][new_vector[point_b]-1]
    obj_function = obj_function + distance_matrix[new_vector[point_a-1]-1][new_vector[point_a]-1] 
    obj_function = obj_function + distance_matrix[new_vector[point_b-1]-1][new_vector[point_b]-1]
    return new_vector[1:-1], obj_function


### Definição dos dados do problema - Matriz de distâncias

In [241]:
matrix = [[0, 12, 3, 23, 1, 5, 32, 56],
        [12, 0, 9, 18, 3, 41, 45, 5],
        [3, 9, 0, 89, 56, 21, 12, 49],
        [23, 18, 89, 0, 87, 46, 75, 17],
        [1, 3, 56, 87, 0, 55, 22, 86],
        [5, 41, 21, 46, 55, 0, 21, 76],
        [32, 45, 12, 75, 22, 21, 0, 11],
        [56, 5, 49, 17, 86, 76, 11, 0]]

matrix

[[0, 12, 3, 23, 1, 5, 32, 56],
 [12, 0, 9, 18, 3, 41, 45, 5],
 [3, 9, 0, 89, 56, 21, 12, 49],
 [23, 18, 89, 0, 87, 46, 75, 17],
 [1, 3, 56, 87, 0, 55, 22, 86],
 [5, 41, 21, 46, 55, 0, 21, 76],
 [32, 45, 12, 75, 22, 21, 0, 11],
 [56, 5, 49, 17, 86, 76, 11, 0]]

In [242]:
def is_symmetric(matrix):
    length = len(matrix)
    for index_1 in range(length-1):
        for index_2 in range(index_1+1,length):
            if matrix[index_1][index_2] != matrix[index_2][index_1]:
                return False
    return True

def check_main_diagonal(matrix):
    length = len(matrix)
    for index in range(length):
        if matrix[index][index] != 0:
            return False
    return True

In [243]:
is_symmetric(matrix), check_main_diagonal(matrix)

(True, True)

### Definição dos dados do problema - conjunto de rotas iniciais para investigação

In [244]:
list_of_routes = [[2,3,8,4,6,7,5],
[8,2,3,4,6,7,5],
[4,2,8,3,6,7,5],
[6,2,8,4,3,7,5],
[7,2,8,4,6,3,5],
[5,2,8,4,6,7,3],
[3,8,2,4,6,7,5],
[3,4,8,2,6,7,5],
[3,6,8,4,2,7,5]]

In [245]:
for route in list_of_routes:
    print(route, calc_tour_length(route, matrix))

[2, 3, 8, 4, 6, 7, 5] 177
[8, 2, 3, 4, 6, 7, 5] 249
[4, 2, 8, 3, 6, 7, 5] 160
[6, 2, 8, 4, 3, 7, 5] 192
[7, 2, 8, 4, 6, 3, 5] 223
[5, 2, 8, 4, 6, 7, 3] 108
[3, 8, 2, 4, 6, 7, 5] 165
[3, 4, 8, 2, 6, 7, 5] 199
[3, 6, 8, 4, 2, 7, 5] 203


#### Teste da vizinhança 2-opt e de funções de perturbação

In [246]:
for route in list_of_routes:
    current_OF_value = calc_tour_length(route, matrix)
    new_route, new_OF_value = local_search_two_opt(route,current_OF_value, matrix)
    assert new_OF_value == calc_tour_length(new_route, matrix)
    print(route,current_OF_value, '---after simple 2-opt local search---',new_route, new_OF_value)

[2, 3, 8, 4, 6, 7, 5] 177 ---after simple 2-opt local search--- [3, 2, 8, 4, 6, 7, 5] 124
[8, 2, 3, 4, 6, 7, 5] 249 ---after simple 2-opt local search--- [2, 8, 3, 4, 6, 7, 5] 245
[4, 2, 8, 3, 6, 7, 5] 160 ---after simple 2-opt local search--- [4, 8, 2, 3, 6, 7, 5] 119
[6, 2, 8, 4, 3, 7, 5] 192 ---after simple 2-opt local search--- [4, 8, 2, 6, 3, 7, 5] 142
[7, 2, 8, 4, 6, 3, 5] 223 ---after simple 2-opt local search--- [2, 7, 8, 4, 6, 3, 5] 209
[5, 2, 8, 4, 6, 7, 3] 108 ---after simple 2-opt local search--- [5, 2, 8, 4, 6, 7, 3] 108
[3, 8, 2, 4, 6, 7, 5] 165 ---after simple 2-opt local search--- [4, 2, 8, 3, 6, 7, 5] 160
[3, 4, 8, 2, 6, 7, 5] 199 ---after simple 2-opt local search--- [2, 8, 4, 3, 6, 7, 5] 188
[3, 6, 8, 4, 2, 7, 5] 203 ---after simple 2-opt local search--- [6, 3, 8, 4, 2, 7, 5] 178


In [247]:
def swap_intraroute(route_vector,point_a,point_b,obj_function,distance_matrix):
    new_vector = [1] + route_vector + [1]
    point_a += 1
    point_b += 1
    # Retirada dos pesos das arestas do trajeto antes da mudança 
    obj_function = obj_function - distance_matrix[new_vector[point_a-1]-1][new_vector[point_a]-1] - distance_matrix[new_vector[point_a]-1][new_vector[point_a+1]-1]
    obj_function = obj_function - distance_matrix[new_vector[point_b-1]-1][new_vector[point_b]-1] - distance_matrix[new_vector[point_b]-1][new_vector[point_b+1]-1]
    # Movimento de mudança do trajeto
    new_vector[point_a], new_vector[point_b] = new_vector[point_b], new_vector[point_a]
    # Adição dos pesos das novas arestas após a mudança
    obj_function = obj_function + distance_matrix[new_vector[point_a-1]-1][new_vector[point_a]-1] + distance_matrix[new_vector[point_a]-1][new_vector[point_a+1]-1]
    obj_function = obj_function + distance_matrix[new_vector[point_b-1]-1][new_vector[point_b]-1] + distance_matrix[new_vector[point_b]-1][new_vector[point_b+1]-1]

    return new_vector[1:-1], obj_function

def generate_random_neighbor_swap_intraroute(solution,objective_function,distance_matrix):
    pointA, pointB = 0, 0
    while pointB == pointA:
        pointA = randint(0,len(solution)-1)
        pointB = randint(0,len(solution)-1)
    solutions_neighbor, objective_function_neighbor = solution.copy(), objective_function
    solutions_neighbor, objective_function_neighbor = swap_intraroute(solution,pointA,pointB,objective_function,distance_matrix)
    return solutions_neighbor, objective_function_neighbor

In [248]:
for route in list_of_routes:
    objective_function = calc_tour_length(route,matrix)
    A, B = generate_random_neighbor_swap_intraroute(route,objective_function,matrix)
    print(A,B,B == calc_tour_length(A,matrix))

[2, 3, 8, 6, 4, 7, 5] 290 True
[8, 2, 5, 4, 6, 7, 3] 233 True
[4, 5, 8, 3, 6, 7, 2] 344 True
[6, 2, 8, 4, 7, 3, 5] 212 True
[8, 2, 7, 4, 6, 3, 5] 305 True
[2, 5, 8, 4, 6, 7, 3] 200 True
[3, 8, 2, 5, 6, 7, 4] 234 True
[3, 4, 8, 7, 6, 2, 5] 186 True
[3, 6, 5, 4, 2, 7, 8] 296 True


#### Funções para implementação da metaheurística ILS

In [249]:
def localSearchProcedure(solution, obj_function, distance_matrix):
    current_solution = solution.copy()
    current_obj_function = obj_function
    while True:
        new_solution, new_obj_function = local_search_two_opt(current_solution,current_obj_function,distance_matrix)
        if new_obj_function < current_obj_function:
            current_solution, current_obj_function = new_solution, new_obj_function
        else:
            return new_solution, new_obj_function

def perturbationProcedure(solution, obj_function, distance_matrix, degree_of_perturbation):
    candidate_solution = solution.copy()
    candidate_obj_function = obj_function
    number_of_vertices = len(distance_matrix)
    amount_of_perturbation = round(degree_of_perturbation*number_of_vertices)
    for iteration in range(amount_of_perturbation):
        candidate_solution, candidate_obj_function = generate_random_neighbor_swap_intraroute(candidate_solution,candidate_obj_function,distance_matrix)
    return candidate_solution, candidate_obj_function

In [250]:
def iteratedLocalSearch(solution, obj_function, distance_matrix, num_iterations = 100, perturbation_degree = 0.4):
    iterations = 0
    current_solution = solution.copy()
    current_obj_function = obj_function

    best_solution = solution.copy()
    best_obj_function = obj_function
    
    while(iterations < num_iterations):
        #Perturbação da solução
        neighbor_solution, neighbor_obj_function = perturbationProcedure(current_solution,current_obj_function,distance_matrix, perturbation_degree)
        
        neighbor_solution, neighbor_obj_function = localSearchProcedure(neighbor_solution,neighbor_obj_function,distance_matrix)
        
        if neighbor_obj_function < 1.1*current_obj_function:
            current_solution = neighbor_solution.copy()
            current_obj_function = neighbor_obj_function
            
            if current_obj_function < best_obj_function:
                best_obj_function = current_obj_function 
                best_solution = current_solution.copy()
        iterations += 1

    return best_solution, best_obj_function

### Execução da metaheurística

In [251]:
for route in list_of_routes:
    print([1] + route + [1], calc_tour_length(route, matrix))

[1, 2, 3, 8, 4, 6, 7, 5, 1] 177
[1, 8, 2, 3, 4, 6, 7, 5, 1] 249
[1, 4, 2, 8, 3, 6, 7, 5, 1] 160
[1, 6, 2, 8, 4, 3, 7, 5, 1] 192
[1, 7, 2, 8, 4, 6, 3, 5, 1] 223
[1, 5, 2, 8, 4, 6, 7, 3, 1] 108
[1, 3, 8, 2, 4, 6, 7, 5, 1] 165
[1, 3, 4, 8, 2, 6, 7, 5, 1] 199
[1, 3, 6, 8, 4, 2, 7, 5, 1] 203


In [252]:
for route in list_of_routes:
    objective_function = calc_tour_length(route,matrix)
    A, B = iteratedLocalSearch(route, objective_function, matrix, num_iterations = 10, perturbation_degree = 0.4)
    print([1] + A + [1], B)

[1, 5, 2, 4, 8, 7, 3, 6, 1] 88
[1, 5, 2, 4, 8, 7, 3, 6, 1] 88
[1, 5, 2, 4, 8, 7, 3, 6, 1] 88
[1, 5, 2, 4, 8, 7, 3, 6, 1] 88
[1, 5, 2, 4, 8, 7, 3, 6, 1] 88
[1, 5, 2, 4, 8, 7, 3, 6, 1] 88
[1, 5, 2, 4, 8, 7, 6, 3, 1] 95
[1, 5, 2, 4, 8, 7, 3, 6, 1] 88
[1, 6, 3, 7, 8, 4, 2, 5, 1] 88


#### Adendo: Implementação da resolução do TSP por *backtracking* e força bruta

In [253]:
import time

list_cities = []

for item in range(len(matrix)):
    list_cities.append(item)

r = len(list_cities)
minimum_path = float('inf')

def display_array(array):
    new_array = []
    for item in array:
        new_array.append(item+1)
    return new_array

def permute(array, start, finish):
    global matrix, minimum_path
    if start == finish: 
        array.append(array[0])
        path_length = makeHamiltonianPath(matrix,array)
        if path_length < minimum_path:
            minimum_path = path_length
            print(display_array(array), '', path_length)
        array.pop()
    else: 
        for i in range(start,finish+1): 
            array[start], array[i] = array[i], array[start] 
            permute(array, start+1, finish) 
            array[start], array[i] = array[i], array[start] # backtracking

def makeHamiltonianPath(matrix,path):
    path_length = 0
    start_point = path[0]
    for item in range (1,len(path)):
        end_point = path[item]
        path_length += matrix[start_point][end_point]
        start_point = end_point
    return path_length

def main():
    time1 = time.time()
    permute(list_cities,0,r-1)
    print("")
    time2 = time.time()
    print("Execution time: ",time2-time1, " seconds")
       
main()

[1, 2, 3, 4, 5, 6, 7, 8, 1]  340
[1, 2, 3, 4, 5, 7, 8, 6, 1]  311
[1, 2, 3, 4, 6, 5, 7, 8, 1]  300
[1, 2, 3, 4, 6, 7, 8, 5, 1]  275
[1, 2, 3, 4, 6, 8, 7, 5, 1]  266
[1, 2, 3, 4, 8, 6, 7, 5, 1]  247
[1, 2, 3, 4, 8, 7, 6, 5, 1]  215
[1, 2, 3, 5, 6, 7, 8, 4, 1]  204
[1, 2, 3, 5, 7, 8, 4, 6, 1]  178
[1, 2, 3, 6, 5, 7, 8, 4, 1]  170
[1, 2, 3, 6, 4, 8, 7, 5, 1]  139
[1, 2, 5, 7, 8, 4, 6, 3, 1]  135
[1, 3, 2, 5, 7, 8, 4, 6, 1]  116
[1, 3, 6, 7, 5, 2, 8, 4, 1]  115
[1, 3, 6, 7, 8, 4, 2, 5, 1]  95
[1, 5, 2, 4, 8, 7, 3, 6, 1]  88

Execution time:  0.32500171661376953  seconds
